1. Thu thập các đường dẫn thuộc mỗi chuyên mục từ trang web https://vnexpress.net/

- Viết hàm lấy tất cả các đường dẫn của bài báo trong trang web

In [18]:
# Gọi thư viện
from urllib.request import urlopen 
from bs4 import BeautifulSoup
import re

# Đường dẫn đến trang web gốc:
URL = 'https://vnexpress.net/'

def _get_links(url):
    links = [] 
    html = urlopen(url).read()  
    soup = BeautifulSoup(html, 'html.parser') 

    # Lấy tất cả đường dẫn trong trang web hiện tại
    urls = set(re.findall(r'https://vnexpress.net/[^\s"]+\.html', str(soup)))  

    for url in urls:
        # Chỉ lấy đường dẫn là trang web nhiều bài báo
        if url != URL and url.startswith(URL) and not url.startswith(URL + '/tac-gia'): 
            links.append(url)

    return links


- Kết quả trả về của hàm với đường dẫn gốc của VNexpress

In [19]:
_get_links(URL)

['https://vnexpress.net/asean-han-quoc-thiet-lap-quan-he-doi-tac-chien-luoc-toan-dien-4802580.html',
 'https://vnexpress.net/dong-thap-cong-bo-tinh-huong-khan-cap-sat-lo-song-tien-4802467.html',
 'https://vnexpress.net/vi-sao-tang-truong-gdp-quy-iii-vuot-moi-du-bao-4802280.html',
 'https://vnexpress.net/70-nam-doi-thay-cua-thu-do-ha-noi-4802219.html',
 'https://vnexpress.net/tien-ve-ha-noi-loi-tien-tri-chien-thang-4802095.html',
 'https://vnexpress.net/cuoc-chien-chinh-tri-trump-harris-giua-mua-mua-bao-4802293.html',
 'https://vnexpress.net/nha-van-han-kang-cua-han-quoc-doat-nobel-van-hoc-2024-4802635.html',
 'https://vnexpress.net/bao-milton-do-bo-florida-suc-gio-193-km-h-4802278-tong-thuat.html',
 'https://vnexpress.net/tac-gia/dang-hung-vo-1124.html',
 'https://vnexpress.net/vang-mieng-sjc-ve-duoi-85-trieu-4802396.html',
 'https://vnexpress.net/bitexco-noi-gi-ve-hon-15-700-ty-dong-nhan-tu-ba-truong-my-lan-4802455.html',
 'https://vnexpress.net/dau-tri-truoc-ca-ghep-dong-thoi-tim-gan

- Lấy đường dẫn của các bài toán thuộc từng chuyên mục

In [ ]:
# Danh sách các chuyên mục
categories = [
    'thoi-su', 'kinh-doanh', 'khoa-hoc', 'giai-tri',
    'the-thao', 'phap-luat', 'giao-duc', 'suc-khoe',
    'doi-song', 'du-lich', 'so-hoa', 'xe'
]

# Danh sách các đường dẫn đến các bài báo của mỗi chuyên mục
article_url = {}

for category in categories:
    article_url[category] = []

    # Lấy 5 trang trong mỗi chuyên mục
    for i in range(1, 6):
        url = '{}/{}-p{}'.format(URL, category, str(i))
        article_url[category] += _get_links(url)

    # Loại bỏ đường dẫn trùng nhau
    article_url[category] = list(set(article_url[category]))

    # Hiển thị số đường dẫn thu thập được trong chuyên mục
    print(category, len(article_url[category]))

# Lấy riêng các đường dẫn trong chuyên mục theo góc nhìn từng mục con
goc_nhin = [
    'binh-luan-nhieu', 'chinh-tri-chinh-sach', 'y-te-su-khoe',
    'kinh-doanh-quan-tri', 'giao-duc-tri-thuc', 'moi-truong',
    'van-hoa-loi-song', 'covid-19'
]

article_url['goc-nhin'] = []

for sub_cate in goc_nhin:
    url = '{}/{}/{}'.format(URL, 'goc-nhin', sub_cate)
    article_url['goc-nhin'] += _get_links(url)

# Loại bỏ đường dẫn trùng nhau trong chuyên mục 'goc-nhin'
article_url['goc-nhin'] = list(set(article_url['goc-nhin']))
print('goc-nhin', len(article_url['goc-nhin']))